# AUEB M.Sc. in Data Science (part-time)

- Semester: Summer 2020

- Course: Big Data Systems and Techniques

- Instructor: Prof. D. Arkoumanis

- Author: Spiros Politis (p3351814)

# Task 2 - Create a parquet file (10%)

Locate the categories of shoes and a create a subset of data with shoes only. Write a spark program that connects to *PostgreSQL DB*, reads the data in DataFrame and writes the *DataFrame* in *HDFS* in *parquet* format.

As deliverable give the above program, the *printSchema* of the *DataFrame* and the *HDFS* *dfs ls* of the directory with the *parquet* files.

## Get data from local PostgreSQL

In [1]:
%%time

query_shoes = "(SELECT DISTINCT * FROM temp_products WHERE LOWER(category_name) LIKE '%shoes%') AS query_shoes"

df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://s01:5432/products") \
    .option("driver","org.postgresql.Driver") \
    .option("dbtable", query_shoes) \
    .option("user", "dinosar") \
    .option("password", "forsharingpurposes") \
    .load()

CPU times: user 0 ns, sys: 3.08 ms, total: 3.08 ms
Wall time: 3.88 s


In [2]:
df.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- upc_id: string (nullable = true)
 |-- descr: string (nullable = true)
 |-- vendor_catalog_url: string (nullable = true)
 |-- buy_url: string (nullable = true)
 |-- manufacturer_name: string (nullable = true)
 |-- sale_price: decimal(38,18) (nullable = true)
 |-- retail_price: decimal(38,18) (nullable = true)
 |-- manufacturer_part_no: string (nullable = true)
 |-- country: string (nullable = true)
 |-- vendor_id: integer (nullable = true)
 |-- category_name: string (nullable = true)
 |-- category_code: string (nullable = true)
 |-- category_id: integer (nullable = true)



In [3]:
%%time

print("DataFrame shape: ({}, {})".format(df.count(), len(df.columns)))

DataFrame shape: (63968, 15)
CPU times: user 4.18 ms, sys: 905 µs, total: 5.09 ms
Wall time: 10.5 s


In [4]:
df.show(5)

+----------+--------------------+------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+-------+---------+-------------+-------------+-----------+
|product_id|                name|upc_id|               descr|  vendor_catalog_url|             buy_url|manufacturer_name|          sale_price|        retail_price|manufacturer_part_no|country|vendor_id|category_name|category_code|category_id|
+----------+--------------------+------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+-------+---------+-------------+-------------+-----------+
|    530395|Gucci Infant's & ...|  null|Double strap desi...|http://www.shopst...|http://www.shopst...|            Gucci|195.0000000000000...|195.0000000000000...|                null|   null|     null|  Boys' Shoes|   boys-shoes|       1599|
|    530396|Haflinger Kids E

## Get data from Google Cloud SQL PostgreSQL

In [5]:
%%time

query_shoes = "(SELECT DISTINCT * FROM temp_products WHERE LOWER(category_name) LIKE '%shoes%') AS query_shoes"

df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://10.120.64.3:5432/products") \
    .option("driver","org.postgresql.Driver") \
    .option("dbtable", query_shoes) \
    .option("user", "postgres") \
    .option("password", "forsharingpurposes") \
    .load()

CPU times: user 2.04 ms, sys: 0 ns, total: 2.04 ms
Wall time: 71.4 ms


In [6]:
df.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- upc_id: string (nullable = true)
 |-- descr: string (nullable = true)
 |-- vendor_catalog_url: string (nullable = true)
 |-- buy_url: string (nullable = true)
 |-- manufacturer_name: string (nullable = true)
 |-- sale_price: decimal(38,18) (nullable = true)
 |-- retail_price: decimal(38,18) (nullable = true)
 |-- manufacturer_part_no: string (nullable = true)
 |-- country: string (nullable = true)
 |-- vendor_id: integer (nullable = true)
 |-- category_name: string (nullable = true)
 |-- category_code: string (nullable = true)
 |-- category_id: integer (nullable = true)



In [7]:
%%time

print("DataFrame shape: ({}, {})".format(df.count(), len(df.columns)))

DataFrame shape: (63968, 15)
CPU times: user 4.01 ms, sys: 872 µs, total: 4.88 ms
Wall time: 13.3 s


In [8]:
df.show(5)

+----------+--------------------+------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+-------+---------+-------------+-------------+-----------+
|product_id|                name|upc_id|               descr|  vendor_catalog_url|             buy_url|manufacturer_name|          sale_price|        retail_price|manufacturer_part_no|country|vendor_id|category_name|category_code|category_id|
+----------+--------------------+------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+-------+---------+-------------+-------------+-----------+
|    530395|Gucci Infant's & ...|  null|Double strap desi...|http://www.shopst...|http://www.shopst...|            Gucci|195.0000000000000...|195.0000000000000...|                null|   null|     null|  Boys' Shoes|   boys-shoes|       1599|
|    530396|Haflinger Kids E

## Write Parquet file

Delete Parquet file if exists.

In [9]:
%%time

!hdfs dfs -rm -r output/query_shoes.parquet

20/07/16 11:43:52 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted output/query_shoes.parquet
CPU times: user 108 ms, sys: 22.2 ms, total: 130 ms
Wall time: 4.33 s


In [10]:
%%time

df.write.parquet("output/query_shoes.parquet")

CPU times: user 1.62 ms, sys: 2.65 ms, total: 4.27 ms
Wall time: 10.5 s


## Read Parquet file

For verification.

In [11]:
%%time

df = spark.read.parquet("output/query_shoes.parquet")

CPU times: user 0 ns, sys: 2.43 ms, total: 2.43 ms
Wall time: 852 ms


In [12]:
df.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- upc_id: string (nullable = true)
 |-- descr: string (nullable = true)
 |-- vendor_catalog_url: string (nullable = true)
 |-- buy_url: string (nullable = true)
 |-- manufacturer_name: string (nullable = true)
 |-- sale_price: decimal(38,18) (nullable = true)
 |-- retail_price: decimal(38,18) (nullable = true)
 |-- manufacturer_part_no: string (nullable = true)
 |-- country: string (nullable = true)
 |-- vendor_id: integer (nullable = true)
 |-- category_name: string (nullable = true)
 |-- category_code: string (nullable = true)
 |-- category_id: integer (nullable = true)



In [13]:
%%time

print("DataFrame shape: ({}, {})".format(df.count(), len(df.columns)))

DataFrame shape: (63968, 15)
CPU times: user 2.49 ms, sys: 0 ns, total: 2.49 ms
Wall time: 734 ms


In [14]:
df.show(5)

+----------+--------------------+------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+-------+---------+-------------+-------------+-----------+
|product_id|                name|upc_id|               descr|  vendor_catalog_url|             buy_url|manufacturer_name|          sale_price|        retail_price|manufacturer_part_no|country|vendor_id|category_name|category_code|category_id|
+----------+--------------------+------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+-------+---------+-------------+-------------+-----------+
|    530395|Gucci Infant's & ...|  null|Double strap desi...|http://www.shopst...|http://www.shopst...|            Gucci|195.0000000000000...|195.0000000000000...|                null|   null|     null|  Boys' Shoes|   boys-shoes|       1599|
|    530396|Haflinger Kids E

---